In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import datetime
from datetime import date

In [ ]:
df = pd.read_csv('/kaggle/input/covid19-indonesia/covid_19_indonesia_time_series_all.csv')
df.head(20)

In [ ]:
df.columns

In [ ]:
print('---Location---')
df.Location.unique()

In [ ]:
date2=[]
for item in df['Date']:
    item2=item.split('/')
    month=int(item2[0])
    day=int(item2[1])
    year=int(item2[2])
    date2+=[datetime.date(year,month,day)]
df['Date']=date2
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df.head()

menghapus kolom dengan semua data bernilai null dan kolom yang tidak dibutuhkan

In [ ]:
data = df.drop(['City or Regency','Name','Item','Kind','Hidden','Location ISO Code',
                'Province','Country','Continent','Island','Time Zone','Special Status',
                'Total Regencies','Total Cities','Total Districts','Total Urban Villages',
                'Total Rural Villages','New Cases per Million','Total Cases per Million',
                'New Deaths per Million','Total Deaths per Million','Case Fatality Rate',
                'Case Recovered Rate','Growth Factor of New Cases','Growth Factor of New Deaths'],axis = 1)
data = data.fillna(0)
data.head(5)

In [ ]:
data.info()

In [ ]:
newest = data.drop_duplicates(subset='Location', keep="last")
newest.head()

# EDA 

Pandemi telah berlangsung selama lebih dari 17 bulan di Indonesia dan hingga saat ini belum menemukan titik terang kapan pandemi akan berakhir. Jika kita lihat dari data yang telah ada sekarang apakah pandemi akan selesai sesegera mungkin ? Data ini akan memberikan gambaran apakah pandemi yang selama ini kita alami dapat terkendali atau justru sebaliknya yang kian memburuk.

## Data Terbaru mengenai COVID 19 Hingga Tanggal 9 Juli 2021

In [ ]:
newest[newest.Location != 'Indonesia'].sort_values(by=['Total Cases'], ascending=False)

## Provinsi dengan Total Kasus Terbanyak

In [ ]:
plt.figure(figsize=(12,9))
plt.bar(newest[newest.Location != 'Indonesia'].sort_values(by=['Total Cases'], ascending=False)['Location'].values[:5], newest[newest.Location != 'Indonesia'].sort_values(by=['Total Cases'], ascending=False)['Total Cases'].values[:5])
plt.title('5 Provinsi Teratas Dengan Total Kasus Paling Banyak', fontsize=14)
plt.xlabel('Provinsi')
plt.show()

In [ ]:
(newest[newest.Location != 'Indonesia'].sort_values(by=['Total Cases'], ascending=False)['Total Cases'].values[:2][0] - newest[newest.Location != 'Indonesia'].sort_values(by=['Total Cases'], ascending=False)['Total Cases'].values[:2][1])/newest[newest.Location != 'Indonesia'].sort_values(by=['Total Cases'], ascending=False)['Total Cases'].values[:2][0] * 100

Chart diatas menunjukan bahwa DKI Jakarta menempati peringkat tertinggi dalam total kasus covid 19. Dibandingkan dengan peringkat 2, DKI jauh mengungguli dengan 30% jumlah kasus dari 1 tingkat dibawahnya yaitu Jawa Barat.

## Provinsi dengan Angka Kematian Terbanyak

In [ ]:
plt.figure(figsize=(12,9))
# plt.bar(newest[newest.Location != 'Indonesia'].sort_values(by=['Total Deaths'], ascending=False)['Location'].values[:5], newest[newest.Location != 'Indonesia'].sort_values(by=['Total Deaths'], ascending=False)['Total Deaths'].values[:5])
# plt.title('5 Provinsi Teratas Dengan Total Kematian Paling Banyak', fontsize=14)
# plt.xlabel('Provinsi')
# plt.show()

sns.barplot(newest[newest.Location != 'Indonesia'].sort_values(by=['Total Deaths'], ascending=False)['Location'].values[:5],  newest[newest.Location != 'Indonesia'].sort_values(by=['Total Deaths'], ascending=False)['Total Deaths'].values[:5])
plt.title('5 Provinsi Teratas Dengan Total Kematian Paling Banyak', fontsize=25)
plt.xlabel('Provinsi', fontsize=15)
plt.show()

## Provinsi dengan Angka Kesembuhan Terbanyak

In [ ]:
plt.figure(figsize=(12,9))
sns.barplot(newest[newest.Location != 'Indonesia'].sort_values(by=['Total Recovered'], ascending=False)['Location'].values[:5],  newest[newest.Location != 'Indonesia'].sort_values(by=['Total Recovered'], ascending=False)['Total Recovered'].values[:5])
plt.title('5 Provinsi Teratas Dengan Total Kesembuhan Paling Banyak', fontsize=25)
plt.xlabel('Provinsi', fontsize=15)
plt.show()

## Provinsi dengan Angka Kasus Aktif Terbanyak

In [ ]:
plt.figure(figsize=(12,9))
sns.barplot(newest[newest.Location != 'Indonesia'].sort_values(by=['Total Active Cases'], ascending=False)['Location'].values[:5],  newest[newest.Location != 'Indonesia'].sort_values(by=['Total Active Cases'], ascending=False)['Total Active Cases'].values[:5])
plt.title('5 Provinsi Teratas Dengan Total Kasus Aktif Paling Banyak', fontsize=25)
plt.xlabel('Provinsi', fontsize=15)
plt.show()

## COVID 19 Di DKI Jakarta

In [ ]:
jakarta = data[data.Location == 'DKI Jakarta'].tail(7).copy()
jakarta

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(jakarta['Date'].values ,jakarta['New Cases'].values)
plt.xlabel('Waktu')
plt.ylabel('Jumlah Kasus Baru')
plt.show()

In [ ]:
print('Persentase Kenaikan dari Tanggal 7-8 sebanyak', (jakarta['New Cases'].values[4:][1]- jakarta['New Cases'].values[4:][0]) / jakarta['New Cases'].values[4:][0] * 100)

Dapat dilihat bahwa grafik menunjukan keadaan yang cukup fluaktif namun ketika dari tanggal 7 ke tanggal 8 mengalami kenaikan yang sangat cukup signifikan sebanyak 38%. Selanjutnya, grafik tetap meningkat hingga ke tgl 9. Bagaimana dengan tingkat kematian serta kesembuhan pada DKI Jakarta selama kurun waktu 6 hari tsb ?

In [ ]:
plt.figure(figsize=(10,9))
plt.plot(jakarta['Date'].values ,jakarta['New Cases'].values, label='Cases')
plt.plot(jakarta['Date'].values, jakarta['New Recovered'].values, label = 'Recover')
plt.plot(jakarta['Date'].values, jakarta['New Deaths'].values, label = 'Death')
plt.title('Perbandingan Peningkatan Kasus Baru, Kematian, dan Kesembuhan. ')
plt.legend()
plt.show()

In [ ]:
# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,9))
# ax1.plot(jakarta['Date'].values, jakarta['New Deaths'].values, label = 'Death')
# ax2.plot(jakarta['Date'].values, jakarta['New Recovered'].values, label = 'Recover')

plt.figure(figsize=(10,9))
plt.plot(jakarta['Date'].values, jakarta['New Deaths'].values, label = 'Death')
plt.title('Angka Kematian Covid 19 DKI Jakarta 3 Juli - 9 Juli 2021')
plt.legend()
plt.show()

Chat diatas menjelaskan bahwa tingkat recover pada DKI Jakarta kian hari kian meningkat dan jumlah yang telah recover pada tangal 9 lebih tingg dari angka kasus baru. Namun pada angka kematian mengalami peningkatan kian harinya yang tiap harinya naik sejumlah 20-40 korban meninggal. 

## COVID 19 Di Jawa Timur

In [ ]:
jatim = data[data.Location == 'Jawa Timur'].tail(7).copy()
jatim

In [ ]:
fig,(ax1,ax2) =  plt.subplots(1,2,figsize=(12,9))
ax1.plot(range(7), jatim['Total Cases'].values)
ax1.set_title('Angka Kasus COVID 19')
ax2.plot(range(7), jatim['Total Deaths'].values)
ax2.set_title('Angka Kematian COVID 19')
plt.suptitle('Total Cases and Deaths', fontsize=20)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,8))
fig.suptitle('Perbandingan Kasus, Kematian, Kesembuhan Pada Awal PPKM', fontsize=20)
ax[0].plot(range(7), jatim['New Cases'].values, label = 'Jawa Timur')
ax[0].plot(range(7), jakarta['New Cases'].values, label = 'DKI Jakarta')
ax[0].set_title('Angka Kasus')
ax[0].legend()
ax[1].plot(range(7), jatim['New Deaths'].values, label = 'Jawa Timur')
ax[1].plot(range(7), jakarta['New Deaths'].values, label = 'DKI Jakarta')
ax[1].set_title('Angka Kematian')
ax[1].legend()
ax[2].plot(range(7), jatim['New Recovered'].values, label = 'Jawa Timur')
ax[2].plot(range(7), jakarta['New Recovered'].values, label = 'DKI Jakarta')
ax[2].set_title('Angka Kesembuhan')
ax[2].legend()
plt.show()

## Prediction

In [ ]:
data = data[['New Cases','New Deaths','New Recovered','New Active Cases','Total Cases','Total Deaths','Total Recovered','Total Active Cases']]
data.tail()

### Feature Selection

In [ ]:
plt.figure(figsize=(10,9))
sns.heatmap(data.corr(), annot=True)
plt.show()

In [ ]:
data = data[['New Cases','New Deaths','New Recovered','Total Cases','Total Deaths','Total Recovered','Total Active Cases']]
data.tail()

### Split Data

In [ ]:
X = data[['New Cases','New Recovered','Total Cases','Total Deaths','Total Recovered','Total Active Cases']]
y = data[['New Deaths']]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.25)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('std_scalar', StandardScaler())
])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lnr = LinearRegression(normalize=True)
lnr.fit(X_train, y_train)

### Model Evaluate

In [ ]:
print(lnr.intercept_)

In [ ]:
coeff_df = pd.DataFrame(lnr.coef_[0],X.columns, columns=['Coefficient'])
coeff_df

### Predict Model

In [ ]:
pred = lnr.predict(X_test)

In [ ]:
plt.scatter(y_test, pred)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [ ]:
test_pred = lnr.predict(X_test)
train_pred = lnr.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

In [ ]:
results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred) , cross_val(LinearRegression())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df

### SVM

In [ ]:
from sklearn.svm import SVR

svr = SVR(C=10, kernel='linear')
svr.fit(X_train, y_train)

In [ ]:
print(svr.intercept_)

In [ ]:
coeff_df = pd.DataFrame(svr.coef_[0],X.columns, columns=['Coefficient'])
coeff_df

In [ ]:
svr_pred = svr.predict(X_test)

In [ ]:
plt.scatter(y_test, svr_pred)

In [ ]:
svr_test_pred = svr.predict(X_test)
svr_train_pred = svr.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, svr_test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, svr_train_pred)

In [ ]:
results_df_2 = pd.DataFrame(data=[["SVR", *evaluate(y_test, svr_test_pred) , cross_val(SVR())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

### Lasso

In [ ]:
from sklearn.linear_model import Lasso

model_lasso = Lasso(alpha=0.1, 
              precompute=True, 
#               warm_start=True, 
              positive=True, 
              selection='random',
              random_state=42)
model_lasso.fit(X_train, y_train)

lasso_test_pred = model_lasso.predict(X_test)
lasso_train_pred = model_lasso.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, lasso_test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, lasso_train_pred)

In [ ]:
results_df_2 = pd.DataFrame(data=[["Lasso Regression", *evaluate(y_test, lasso_test_pred) , cross_val(Lasso())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=1000)
rf_reg.fit(X_train, y_train)

test_pred = rf_reg.predict(X_test)
train_pred = rf_reg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)

print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

In [ ]:
plt.scatter(y_test, test_pred)

In [ ]:
results_df_2 = pd.DataFrame(data=[["RF Regression", *evaluate(y_test, test_pred) , cross_val(RandomForestRegressor())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

In [ ]:
results_df.set_index('Model', inplace=True)
results_df['R2 Square'].plot(kind='barh', figsize=(12,9))